In [9]:
import tensorflow as tf
import pathlib
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
#path dataset dan parameter
base_dir = pathlib.Path(".") 
data_split_dir = base_dir / 'dataset_final'

img_height = 224
img_width = 224
batch_size = 32
seed = 42
autotune = tf.data.AUTOTUNE

NUM_CLASSES = 4  # Jumlah kelas untuk klasifikasi aflatoksin

In [11]:
# --- HEADER UTAMA ---
print("==================PROSES PEMUATAN DATASET==================")
# 1. Load Train Dataset
print(f"\n[1/3] Memuat {data_split_dir / 'train'}")
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_split_dir / 'train',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    seed=seed
)

# 2. Load Validation Dataset
print(f"\n[2/3] Memuat {data_split_dir / 'val'}")
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_split_dir / 'val',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False, 
    seed=seed
)

# 3. Load Test Dataset
print(f"\n[3/3] Memuat {data_split_dir / 'test'}")
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_split_dir / 'test',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    seed=seed
)

# --- Informasi Kelas & Mapping ---
class_names = train_ds.class_names

print(f"\n==================INFORMASI KELAS & MAPPING==================")
print(f"Kelas ditemukan: {class_names}")

# Menampilkan Tabel Mapping yang Rapi
print(f"\n{'Index Model':<15} | {'Nama Folder':<15} | {'Nilai PPB (Target)':<20}")
print("-" * 60)
for idx, name in enumerate(class_names):
    print(f"{idx:<15} | {name:<15} | {name:<25}")

print(f"\nSelesai. Dataset siap digunakan.")

==================PROSES PEMUATAN DATASET==================

[1/3] Memuat dataset_final\train
Found 1200 files belonging to 4 classes.



[2/3] Memuat dataset_final\val
Found 87 files belonging to 4 classes.

[3/3] Memuat dataset_final\test
Found 90 files belonging to 4 classes.

==================INFORMASI KELAS & MAPPING==================
Kelas ditemukan: ['1', '2', '3', '4']

Index Model     | Nama Folder     | Nilai PPB (Target)  
------------------------------------------------------------
0               | 1               | 1                        
1               | 2               | 2                        
2               | 3               | 3                        
3               | 4               | 4                        

Selesai. Dataset siap digunakan.


In [12]:
print("Step 2: Setting up data augmentation...")

# Data augmentation using Keras layers (TensorFlow 2.x compatible)
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.028),  # 0.1 radians ≈ 5.7 degrees (smaller rotation)
    # tf.keras.layers.RandomBrightness(0.1),
    # tf.keras.layers.RandomContrast(0.1)
], name="data_augmentation")

print("✓ RandomFlip (horizontal)")
print("✓ RandomRotation (±10°)")

# Preprocessing function with Keras augmentation
def preprocess_with_augmentation(images, labels, training=False):
    # Keep images in 0-255 range (ResNet50 preprocess_input expects 0-255)
    images = tf.cast(images, tf.float32)
    
    if training:
        # Apply augmentation (hanya untuk training)
        images = data_augmentation(images, training=True)
    
    # Normalisasi imageNet wajib, fungsi ini mengambil input RGB dan normalisasi ke range yg digunakan ResNet
    images = preprocess_input(images)
    
    # Labels tetap integer untuk sparse_categorical_crossentropy
    return images, labels

# Apply preprocessing
train_ds = train_ds.map(lambda x, y: preprocess_with_augmentation(x, y, training=True), 
                       num_parallel_calls=autotune)
val_ds = val_ds.map(lambda x, y: preprocess_with_augmentation(x, y, training=False), 
                   num_parallel_calls=autotune)
test_ds = test_ds.map(lambda x, y: preprocess_with_augmentation(x, y, training=False), 
                     num_parallel_calls=autotune)

print("✓ Preprocessing applied to all datasets")

Step 2: Setting up data augmentation...
✓ RandomFlip (horizontal)
✓ RandomRotation (±10°)
✓ Preprocessing applied to all datasets
